In [4]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
from imblearn.over_sampling import RandomOverSampler


# 데이터 로딩
mbti_data = pd.read_csv('./MBTI 500.csv')

# NLTK 라이브러리의 불용어 데이터 다운로드
nltk.download('punkt')
nltk.download('stopwords')

# 불용어 리스트
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # 소문자 변환
    text = text.lower()

    # 특수 문자 제거
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)

    # 토큰화 (단어 단위로 분리)
    tokens = word_tokenize(text)

    # 불용어 제거
    filtered_tokens = [word for word in tokens if word not in stop_words]

    # 정제된 텍스트를 문자열로 재결합
    text = ' '.join(filtered_tokens)

    return text

# 모든 게시물에 대해 전처리 수행
mbti_data['posts'] = mbti_data['posts'].apply(preprocess_text)
# TF-IDF 계산
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X = tfidf_vectorizer.fit_transform(mbti_data['posts'])

# TF-IDF 벡터라이저 저장
joblib.dump(tfidf_vectorizer, 'tfidf_vectorizer.joblib')

y = mbti_data['type']

# 학습 데이터와 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# 오버샘플링을 위한 인스턴스 생성
ros = RandomOverSampler(random_state=42)

# 오버샘플링 전 학습 데이터의 클래스 분포 확인
print("오버샘플링 전 학습 데이터 클래스 분포:")
print(pd.Series(y_train).value_counts())

# 오버샘플링 적용
X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)

# 오버샘플링 후 학습 데이터의 클래스 분포 확인
print("\n오버샘플링 후 학습 데이터 클래스 분포:")
print(pd.Series(y_train_resampled).value_counts())

# 모델 학습
model = MultinomialNB()
model.fit(X_train_resampled, y_train_resampled)

# 모델 평가
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\JG\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\JG\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


오버샘플링 전 학습 데이터 클래스 분포:
INTP    22448
INTJ    20126
INFJ    13493
INFP    10938
ENTP    10558
ENFP     5522
ISTP     3105
ENTJ     2677
ESTP     1795
ENFJ     1364
ISTJ     1126
ISFP      806
ISFJ      582
ESTJ      432
ESFP      322
ESFJ      166
Name: type, dtype: int64

오버샘플링 후 학습 데이터 클래스 분포:
INTP    22448
INFP    22448
ENFP    22448
ISTP    22448
INFJ    22448
ISTJ    22448
INTJ    22448
ENTP    22448
ESTP    22448
ENTJ    22448
ISFJ    22448
ENFJ    22448
ISFP    22448
ESFP    22448
ESTJ    22448
ESFJ    22448
Name: type, dtype: int64
              precision    recall  f1-score   support

        ENFJ       0.40      0.71      0.51       170
        ENFP       0.55      0.73      0.62       645
        ENTJ       0.70      0.78      0.74       278
        ENTP       0.73      0.69      0.71      1167
        ESFJ       0.32      0.40      0.35        15
        ESFP       0.28      0.50      0.36        38
        ESTJ       0.81      0.78      0.80        50
        ESTP       0.6

In [5]:
import joblib

# 학습된 모델을 파일로 저장
model_filename = 'mbti_model.joblib'
joblib.dump(model, model_filename)

['mbti_model.joblib']

## 개별 MBTI 유형별 분석

Precision은 모델이 해당 유형이라고 예측한 경우 중 실제로 해당 유형인 비율입니다.
ESTJ (0.81), INTJ (0.80), INTP (0.78), ENTP (0.73), INFJ (0.71) 유형에서 상대적으로 높은 Precision 값을 보입니다. 이는 이러한 유형에 대한 예측이 정확하다는 것을 나타냅니다.
Recall:

Recall은 실제 해당 유형인 경우 중 모델이 올바르게 예측한 비율입니다.
ESTP (0.88), ISTJ (0.79), ENTJ (0.78), ENFP (0.73), ESFP (0.50) 유형에서 상대적으로 높은 Recall 값을 보입니다. 이는 이러한 유형을 잘 감지한다는 것을 의미합니다.
F1-Score:

F1-Score는 Precision과 Recall의 조화 평균으로, 두 지표의 균형을 나타냅니다.
ESTJ (0.80), ENTJ (0.74), ESTP (0.73), ENTP (0.71), INTP (0.72) 유형이 높은 F1-Score를 보이며, 이는 균형 잡힌 예측 성능을 나타냅니다.
전체적인 모델 성능
Accuracy: 전체 데이터 중 모델이 올바르게 예측한 비율은 0.68(68%)입니다. 이는 전체적으로 괜찮은 성능을 나타냅니다.
Macro Avg: 모든 클래스에 대한 평균 Precision, Recall, F1-Score 입니다. 여기서는 각각 0.55, 0.69, 0.59로 나타났습니다. 특히 Recall이 높게 나온 것은 모델이 다양한 유형을 잘 감지하지만, 실제로 그 유형이 아닌 경우에도 자주 그 유형으로 분류한다는 것을 의미할 수 있습니다.
Weighted Avg: 클래스의 샘플 크기를 고려한 가중 평균 성능입니다. Precision, Recall, F1-Score가 각각 0.71, 0.68, 0.69로 나타났습니다. 이는 데이터셋의 불균형을 고려한 평균 성능을 나타내며, 전반적으로 균형 잡힌 성능을 보여줍니다.


해석
이 결과는 모델이 특정 MBTI 유형에 대해 잘 예측하는 경향이 있으며, 특히 일부 유형에서 높은 정확도를 보임을 나타냅니다. 그러나 일부 유형에서 낮은 Precision이나 Recall 값을 보여, 이 유형에 대한 예측 개선이 필요함을 나타냅니다. 또한, 전체적인 성능 지표는 모델이 균형 잡힌 예측을 하고 있음을 보여줍니다. 하지만 여전히 개선의 여지가 있으며,특히 소수 클래스에 대한 예측력을 향상시키기 위한 방법을 고려할 필요가 있습니다. 소수 클래스의 경우, 낮은 Precision과 Recall 값이 나타나는 경향이 있으므로, 이러한 유형들에 대한 예측 정확도를 개선하는 것이 중요합니다.



## 모델의 성능을 높히기 위해서 LinearSVC 모델로 한번 더 학습

In [6]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
from imblearn.over_sampling import RandomOverSampler
from sklearn.svm import LinearSVC
import joblib

# 데이터 로딩
mbti_data = pd.read_csv('./MBTI 500.csv')

# NLTK 라이브러리의 불용어 데이터 다운로드
nltk.download('punkt')
nltk.download('stopwords')

# 불용어 리스트
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # 소문자 변환
    text = text.lower()

    # 특수 문자 제거
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)

    # 토큰화 (단어 단위로 분리)
    tokens = word_tokenize(text)

    # 불용어 제거
    filtered_tokens = [word for word in tokens if word not in stop_words]

    # 정제된 텍스트를 문자열로 재결합
    text = ' '.join(filtered_tokens)

    return text

# 모든 게시물에 대해 전처리 수행
mbti_data['posts'] = mbti_data['posts'].apply(preprocess_text)
# TF-IDF 계산
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X = tfidf_vectorizer.fit_transform(mbti_data['posts'])

# TF-IDF 벡터라이저 저장
joblib.dump(tfidf_vectorizer, 'tfidf_vectorizer.joblib')

y = mbti_data['type']

# 학습 데이터와 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# 오버샘플링을 위한 인스턴스 생성
ros = RandomOverSampler(random_state=42)

# 오버샘플링 전 학습 데이터의 클래스 분포 확인
print("오버샘플링 전 학습 데이터 클래스 분포:")
print(pd.Series(y_train).value_counts())

# 오버샘플링 적용
X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)

# 오버샘플링 후 학습 데이터의 클래스 분포 확인
print("\n오버샘플링 후 학습 데이터 클래스 분포:")
print(pd.Series(y_train_resampled).value_counts())

# 모델 학습
clf = LinearSVC()
clf.fit(X_train_resampled, y_train_resampled)

# 모델 평가
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

# 학습된 모델을 파일로 저장
model_filename = 'mbti_model_2.joblib'
joblib.dump(model, model_filename)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\JG\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\JG\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


오버샘플링 전 학습 데이터 클래스 분포:
INTP    22448
INTJ    20126
INFJ    13493
INFP    10938
ENTP    10558
ENFP     5522
ISTP     3105
ENTJ     2677
ESTP     1795
ENFJ     1364
ISTJ     1126
ISFP      806
ISFJ      582
ESTJ      432
ESFP      322
ESFJ      166
Name: type, dtype: int64

오버샘플링 후 학습 데이터 클래스 분포:
INTP    22448
INFP    22448
ENFP    22448
ISTP    22448
INFJ    22448
ISTJ    22448
INTJ    22448
ENTP    22448
ESTP    22448
ENTJ    22448
ISFJ    22448
ENFJ    22448
ISFP    22448
ESFP    22448
ESTJ    22448
ESFJ    22448
Name: type, dtype: int64
              precision    recall  f1-score   support

        ENFJ       0.59      0.71      0.64       170
        ENFP       0.69      0.78      0.73       645
        ENTJ       0.66      0.84      0.74       278
        ENTP       0.79      0.81      0.80      1167
        ESFJ       0.38      0.53      0.44        15
        ESFP       0.60      0.55      0.58        38
        ESTJ       0.71      0.90      0.80        50
        ESTP       0.8

['mbti_model_2.joblib']

## Multinomial Naive Bayes (MultinomialNB):

이 모델은 텍스트 데이터와 같은 이산적 특징에 잘 작동하는 확률 기반의 분류기입니다.
Naive Bayes 분류기는 특징 간의 독립성을 가정합니다. 즉, 한 특징이 주어졌을 때 다른 특징들의 영향을 받지 않는다고 가정합니다.
작은 데이터셋에도 잘 작동하지만, 복잡한 패턴이나 특징 간의 상호작용을 모델링하는 데는 제한적일 수 있습니다.
이 분류기는 각 클래스에 대한 사전 확률을 기반으로 하며, 데이터의 분포가 각 클래스의 사전 확률과 얼마나 일치하는지에 따라 예측을 합니다.

## Linear Support Vector Classification (LinearSVC):

이 모델은 서포트 벡터 머신 (SVM)을 기반으로 하는 분류기입니다.
SVM은 특징 공간에서 클래스 간의 최대 마진을 찾아내는 것을 목표로 합니다. 즉, 클래스를 구분하는 결정 경계를 찾는 데 초점을 맞춥니다.
LinearSVC는 특히 텍스트 분류와 같은 고차원 데이터에 효과적으로 작동하며, 복잡한 패턴과 특징 간의 상호작용을 더 잘 포착할 수 있습니다.
이 모델은 특히 데이터가 선형적으로 구분될 수 있을 때 강력한 성능을 발휘합니다.
성능 차이의 이유:

MultinomialNB는 간단한 확률적 접근을 사용하여 클래스를 예측하는 반면, LinearSVC는 데이터의 구조를 더 복잡하게 분석하여 클래스를 구분합니다.
텍스트 데이터의 경우, 특히 고차원에서는 LinearSVC가 MultinomialNB보다 더 정확한 결정 경계를 찾을 가능성이 높습니다. 이는 텍스트 데이터에 내재된 복잡한 패턴과 상호 의존성을 더 잘 포착하기 때문입니다.
LinearSVC는 일반적으로 Naive Bayes보다 더 강력한 성능을 발휘하는 경향이 있으며, 이는 데이터셋의 특성과 복잡성에 따라 달라질 수 있습니다.
결론적으로, LinearSVC가 MultinomialNB보다 더 높은 정밀도, 재현율, F1 점수를 달성한 것은 LinearSVC가 복잡한 텍스트 데이터 패턴을 더 잘 분석하고 예측하기 때문으로 볼 수 있습니다.